<a href="https://colab.research.google.com/github/Jacob-Thomas1/Jacob/blob/master/LiveBuildingSystemJacob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SQLite Setup

In [ ]:
import random
import sqlite3
import datetime
import time
import json

In [ ]:
import sqlite3
conn = sqlite3.connect('database.db')

## Creat Meters and Meter_data tables

In [ ]:
conn.execute('CREATE TABLE meters (id INT, label VARCHAR(35), PRIMARY KEY (id))')
print("Creating 'meters' Database...")

In [ ]:
conn.execute('CREATE TABLE meter_data (id INT, meter_id INT, timestamp DATATIME, value INT, PRIMARY KEY (id), FOREIGN KEY(meter_id) REFERENCES meters(id))')
print("Creating 'meter_data' Database...")

## Adding data to Meter table

In [ ]:
def insert_vals_meters(id_p, label_p): 
  with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            query = f"INSERT INTO meters (id, label) VALUES ({id_p},'{label_p}');"         
            cur.execute(query)
            print("Added", id_p, "to DB")

In [ ]:
list_id = []
for i in range(0, 5): 
  id = random.randint(1000,5000)
  list_id.append(id)

  label = "meter_" + chr(97+i)

  insert_vals_meters(id, label)

## Select / Executing a Query

In [ ]:
def display_query(cur):
  l = cur.fetchall()
  for ele in l:
    print(ele)

In [ ]:
cur = conn.execute('SELECT * FROM meters')
display_query(cur)

## Adding data to meter_data table 

In [ ]:
def insert_vals_meter_id(id_p, meter_id, value): 
  with sqlite3.connect('database.db') as con:
            cur = con.cursor()
            
            curr_time = datetime.datetime.utcnow()
            timestamp = curr_time.strftime('%Y-%m-%d %H:%M:%S')

            query = f"INSERT INTO meter_data (id, meter_id, timestamp, value) VALUES ({id_p}, {meter_id}, '{timestamp}', {value});"         
            cur.execute(query)
            print("Added", id_p, "to meter_id DB")

In [ ]:
for i in range(0, 5): 

  for j in range(0, 5): 
    id = random.randint(0,1000000000)
    insert_vals_meter_id(id, list_id[i], 1000 + j)
    time.sleep(5)

In [ ]:
def display_query(cur):
  l = cur.fetchall()
  for ele in l:
    print(ele)

In [ ]:
cur = conn.execute('SELECT * FROM meter_data')
display_query(cur)

## Confirmation of join between tables

In [ ]:
cur = conn.execute("""SELECT m.id, m.label, md.timestamp, md.value
                      FROM meters m INNER JOIN meter_data md
                      ON m.id = md.meter_id
                      WHERE m.id = 1929
                      ORDER BY md.timestamp
                      """)
display_query(cur)

## Setting up Ngrok Server Hosting

In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok

!ngrok authtoken "ADD YOUR NGROK TOKEN" 

In [ ]:
!mkdir templates

## Run to Host final app

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template
from flask import request

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/')
def running_server():
    return render_template("home.html")

@app.route('/meters')
def list():
   con = sqlite3.connect('database.db')
   con.row_factory = sqlite3.Row
   
   cur = con.cursor()
   cur.execute("SELECT * FROM meters")
   
   rows = []

   l = cur.fetchall()

   return render_template("list.html",rows = l)

@app.route('/meters/<id>')
def query(id):

   con = sqlite3.connect('database.db')
   con.row_factory = sqlite3.Row
   
   cur = con.cursor()
   cur.execute(f"""SELECT m.id, m.label, md.timestamp, md.value
                      FROM meters m INNER JOIN meter_data md
                      ON m.id = md.meter_id
                      WHERE m.id = {id}
                      ORDER BY md.timestamp""")


   rows = cur.fetchall()

   
   return json.dumps( [dict(row) for row in rows] ) 


app.run()